
# transfer_fs
### Import Python Packages


In [1]:
from tensorflow.python.keras.backend import clear_session
from tensorflow.python.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.python.keras.utils.layer_utils import print_summary
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.models import load_model
import numpy as np
from CLRNet_convlstm import CLRNet
from Utility_functions import create_sequence
from datetime import datetime as dt
from DFVDSequence import DFVDSequence
import matplotlib.pyplot as plt
print("yolo")
from sklearn.metrics import classification_report
import pandas as pd
# import shutil

# from src.xception_convlstm import Xception
# print(cv2.__version__)
# import random
# random.seed(32)

dataset_dir='DeepFakeDatasetReal'
# from src.cl_basic import cl_basic
import ipykernel
import os
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras import backend as K
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
 
# Choose GPU NUMBERS [0, 1, 2, 3]
os.environ["CUDA_VISIBLE_DEVICES"]="0"
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
K.set_session(sess)
sess.run(tf.global_variables_initializer())

# Load DeepFake Transfer Dataset

In [2]:
train_video_per_batch=16
val_video_per_batch=16
test_video_per_batch=16
frames_per_video=80 #total frames
frames_per_video_per_batch=5 #frames in one batch
image_size=240
channel=3

X_train,y_train, class_weights_train =create_sequence('datasets/TransferLearning/DeepFake/train/',frames_per_video_per_batch,frames_per_video)
X_val,y_val,class_weights_val=create_sequence('datasets/TransferLearning/DeepFake/val/',frames_per_video_per_batch,frames_per_video)
train_it=DFVDSequence(X_train,y_train,train_video_per_batch,frames_per_video_per_batch,image_size,True)
val_it=DFVDSequence(X_val,y_val,val_video_per_batch,frames_per_video_per_batch,image_size,False)

fig=plt.figure(figsize=(10, 100))
columns = 4
rows = 23
print("Training")
x=1
temp=train_it.__getitem__(1)
fig.add_subplot(rows, columns, x)
print("Min:",np.array(temp[0][0,0]).min(),"Max:",np.array(temp[0][0,0]).max())
print("Min:",np.array(temp[0][1,0]).min(),"Max:",np.array(temp[0][1,0]).max())
plt.imshow(np.array(temp[0][0,0]))
fig.add_subplot(rows, columns, x+1)
plt.imshow(np.array(temp[0][1,0]))
print(temp[0].shape,temp[1].shape,train_it.dataset_size,train_it.frame_counter,train_it.__len__(),temp[1][0],temp[1][1])
# x+=2
plt.title("Training Set Examples")        
plt.show()
train_it.on_epoch_end()

fig=plt.figure(figsize=(10, 100))
columns = 4
rows = 23
print("Validation")
x=1
temp=val_it.__getitem__(1)
fig.add_subplot(rows, columns, x)
print("Min:",np.array(temp[0][0,0]).min(),"Max:",np.array(temp[0][0,0]).max())
print("Min:",np.array(temp[0][1,0]).min(),"Max:",np.array(temp[0][1,0]).max())
plt.imshow(np.array(temp[0][0,0]),)
fig.add_subplot(rows, columns, x+1)
plt.imshow(np.array(temp[0][1,0]))
print(temp[0].shape,temp[1].shape,val_it.dataset_size,val_it.frame_counter,val_it.__len__(),temp[1][0],temp[1][1])
plt.title("Validation Set Examples")   
plt.show()
val_it.on_epoch_end()

# Load Model

In [3]:
# model=CLRNet(input_shape=(frames_per_video_per_batch,image_size, image_size, channel), classes=2, block='bottleneck', residual_unit='v2',
#            repetitions=[3, 4, 6, 3], initial_filters=64, activation='softmax', include_top=False,
#            input_tensor=None, dropout=0.25, transition_dilation_rate=(1, 1),
#            initial_strides=(2, 2), initial_kernel_size=(7, 7), initial_pooling='max',
#            final_pooling=None, top='classification')
# print_summary(model, line_length=150, positions=None, print_fn=None)
model = load_model('models/FaceSwap/CLRNet50(bk)/19-12-17-10_03/CLRNet50(bk)_19-12-17-10_03-130-0.99.hdf5')

In [17]:
# layers = [(layer, layer.name, layer.trainable) for layer in model.layers]
# count =0
# print(len(model.layers))
# for layer in model.layers:
#     count+=1
#     if count==len(model.layers):
#         continue
#     else:
#         layer.trainable = False
# print(layers[206])


# # Keep a copy of the original class
# BatchNormalization = layers.BatchNormalization

# # Patch the class temporarily
# layers.BatchNormalization = FrozenBatchNormalization

# set_trainable = False
# for layer in model.layers:

#     if layer.name in ['res5c_branch2b', 'res5c_branch2c', 'activation_21']:
#         set_trainable = True
#     if set_trainable:
#         layer.trainable = True
#     else:
#         layer.trainable = False
#     if 'batch_normalization' in layer.name:
#         print(layer.name)
#         layer.trainable = False
from tensorflow.python.keras import layers
class FrozenBatchNormalization(layers.BatchNormalization):
    def call(self, inputs, training=None):
        return super().call(inputs=inputs, training=False)
mode ='training'
top_k_layers=140

# model.trainable = True
if mode == 'training':
    _bottom_layers = model.layers[:-top_k_layers]
    _top_layers = model.layers[-top_k_layers:]
elif mode == 'inference':
    _bottom_layers = model.layers
    _top_layers = []

for _layer in _bottom_layers:
    _layer.trainable = False
    if 'batch_normalization' in _layer.name:
        print('Freezing BN layers ... {}'.format(_layer.name))
        _layer = FrozenBatchNormalization

for _layer in _top_layers:
    _layer.trainable = True
    if 'batch_normalization' in _layer.name:
        print('Unfreezing BN layers ... {}'.format(_layer.name))
        _layer = layers.BatchNormalization
# for layer in model.layers:
#     if 'batch_normalization' in layer.name:
#         print(layer)
layers_df = [(layer, layer.name, layer.trainable) for layer in model.layers]
pd.DataFrame(layers_df, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])

# print(df.head(207))

In [4]:
print_summary(model, line_length=150, positions=None, print_fn=None)
adam_fine = Adam(lr=0.00005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss='binary_crossentropy',
                  optimizer=adam_fine,
                  metrics=['accuracy'])

_id = dt.now().strftime("%y-%m-%d-%H_%M")
save_dir=os.path.join(os.getcwd(),'models','transfer_fs','CLRNet','DeepFake',_id)
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

filepath=os.path.join(save_dir,"{epoch:02d}-{acc:.2f}"+".hdf5")
checkpoint = ModelCheckpoint(filepath)#, monitor='acc', verbose=2, save_best_only=True, mode='max')
csv_logger = CSVLogger(os.path.join(save_dir,"training.csv"), append=True, separator=',')
# earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
# mcp_save = ModelCheckpoint('.mdl_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')
# reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, epsilon=1e-4, mode='min')


# earlyStop=EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto', baseline=None, restore_best_weights=False)
# callbacks_list = [csv_logger]
callbacks_list = [checkpoint,csv_logger]

In [5]:
model.fit_generator(generator=train_it, validation_data=val_it,epochs=100,
                    callbacks=callbacks_list,shuffle=False,class_weight=class_weights_train,verbose=1)

In [ ]:
model.fit_generator(generator=train_it, validation_data=val_it,epochs=150,initial_epoch=100,
                    callbacks=callbacks_list,shuffle=False,class_weight=class_weights_train,verbose=1)

In [12]:
mode ='inference'
top_k_layers=140

# model.trainable = True
if mode == 'training':
    _bottom_layers = model.layers[:-top_k_layers]
    _top_layers = model.layers[-top_k_layers:]
elif mode == 'inference':
    _bottom_layers = model.layers
    _top_layers = []

for _layer in _bottom_layers:
    _layer.trainable = False
    if 'batch_normalization' in _layer.name:
        print('Freezing BN layers ... {}'.format(_layer.name))
        _layer = FrozenBatchNormalization

for _layer in _top_layers:
    _layer.trainable = True
    if 'batch_normalization' in _layer.name:
        print('Unfreezing BN layers ... {}'.format(_layer.name))
        _layer = layers.BatchNormalization
# for layer in model.layers:
#     if 'batch_normalization' in layer.name:
#         print(layer)
layers_df = [(layer, layer.name, layer.trainable) for layer in model.layers]
pd.DataFrame(layers_df, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])

In [5]:
predictions = model.predict_generator(FaceSwap_it, verbose=1)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = FaceSwap_it.pre_classes

In [6]:
# class_labels = list(FaceSwap_it.class_indices.keys()) 
report = classification_report(true_classes, predicted_classes, digits=4)
print(report)

In [ ]:
# %load_ext autoreload
# %autoreload 2

In [6]:

print(FaceSwap_it.pre_classes)
# print(len(predictions))
# print(len(y_FaceSwap))

# DeepFakes Test

### Load DeepFakes Test Dataset

In [13]:
frames_per_video_per_batch=5
frames_per_video=80
image_size=240
test_video_per_batch=16
data_augmentation=False
X_DeepFake,y_DeepFake,class_weights_DeepFake=create_sequence('datasets/DeepFake/test/',frames_per_video_per_batch,frames_per_video)
DeepFake_it=DFVDSequence(X_DeepFake,y_DeepFake,test_video_per_batch,frames_per_video_per_batch,image_size,data_augmentation)

### Predict DeepFakes

In [14]:
predictions = model.predict_generator(DeepFake_it, verbose=1)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = DeepFake_it.pre_classes
# class_labels = list(DeepFake_it.class_indices.keys()) 
report = classification_report(true_classes, predicted_classes,  digits=4)
print(report)

# FaceSwap Test

### Load FaceSwap Test Dataset

In [15]:
frames_per_video_per_batch=5
frames_per_video=80
image_size=240
test_video_per_batch=16
data_augmentation=False

X_FaceSwap,y_FaceSwap,class_weights_FaceSwap=create_sequence('datasets/FaceSwap/test/',frames_per_video_per_batch,frames_per_video)
FaceSwap_it=DFVDSequence(X_FaceSwap,y_FaceSwap,test_video_per_batch,frames_per_video_per_batch,image_size,data_augmentation)


### Predict FaceSwap

In [16]:
predictions = model.predict_generator(FaceSwap_it, verbose=1)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = FaceSwap_it.pre_classes
# class_labels = list(FaceSwap_it.class_indices.keys()) 
report = classification_report(true_classes, predicted_classes, digits=4)
print(report)

# Face2Face Test

### Load Face2Face Test Dataset

In [11]:
frames_per_video_per_batch=5
frames_per_video=80
image_size=240
test_video_per_batch=16
data_augmentation=False
X_Face2Face,y_Face2Face,class_weights_Face2Face=create_sequence('datasets/Face2Face/test/',frames_per_video_per_batch,frames_per_video)
Face2Face_it=DFVDSequence(X_Face2Face,y_Face2Face,test_video_per_batch,frames_per_video_per_batch,image_size,data_augmentation)

### Predict Face2Face

In [12]:
predictions = model.predict_generator(Face2Face_it, verbose=1)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = Face2Face_it.pre_classes
# class_labels = list(Face2Face_it.class_indices.keys()) 
report = classification_report(true_classes, predicted_classes, digits=4)
print(report)